9.1 Compounding returns r1--> single period. R1--> gross

In [ ]:
# 1 way
(df
   # compute gross returns for each asset
   .assign(R = 1+df['ret'])
   # for each portfolio and time period...
   .groupby(['asset','timeperiod'])
   # get the gross returns, and cumulate by taking the product
   ['R'].prod()
   # subtract one to get back to simple returns
   -1
)

In [ ]:
#2nd way
(df
 # for each asset and time period
 .groupby(["asset", "timeperiod"])
 # agg() does the function inside for each group (asset-timeperiod combo)
 # 1+x['ret'] gets gross return (R) for each observation in the data
 # (1+x{'ret']).prod() takes the product of all R for that group
 # and then subtract one
 .agg(ret=lambda x: ((1 + x["ret"]).prod()) - 1)
)

In [ ]:
# 3 way
(df
 .assign(logR = np.log(1+df.ret))
 # for each asset and time period
 .groupby(["asset", "timeperiod"])
 # sum log returns
 ['logR'].sum()
 # exponentiate and subtract 1
 .assign(ret = lambda x: np.exp(x['logR'])-1
)

Explaination for 3
1. Why we have to use log?
Adding returns over time. Log returns can be summed across time in order to get the toal log return since simple returns can not be added over time.
2. Steps
A.Calculate log returns for each period
B. Sum them over the periods
C.Exponentiate the sum

9.2 Rolling Returns
Rolling returns. Rolling returns are used to calculate the return of an asset over a moving time window. Since some data is not perfect, might be missing


In [ ]:
Questions: Stock markets close on weekends, so we can’t always just take the last 5 rows.

In [ ]:
df['ret1week'] = (
    df.set_index('Date')
    .groupby('Firm')['ret']
    .rolling('7D', 1)
    .apply(lambda x: np.prod(1 + x) - 1)
    .values
)

# make sure the data is used for rolling, do the calculation, look at pervious 7 calendar days, compute the retuns,ensure resuly lines up as column.


9.3 Expanding returns
Since we need the total returns from day 0 to day n+1, this is not a fixed number of units, so we need to expand it.

In [ ]:
# Getting the expanding returns 2 ways

#1.
df.assign(R=1+df['r']).groupby('firm')['R'].cumprod()

#2.
df.groupby('firm')['r'].expanding().apply(lambda x: np.prod(1+x))

Might be very useful!!! 9.6 Cross-Sectional / “Assaying Anomalies” Style Analysis

In [ ]:
#STEP 1 Get the factor portfolio returns
# I think this are in the opensourceAP dataset, and you should grab from that source instead, IMO
df_factors = ff.FamaFrenchReader('F-F_Research_Data_5_Factors_2x3', start='1900-01-01').read()[0]

# add momentum to this
mom = ff.FamaFrenchReader('F-F_Momentum_Factor', start='1900-01-01').read()[0] # add momentum
mom.columns = ['Mom'] # rename
df_factors = pd.merge(df_factors, mom, left_index=True, right_index=True)
df_factors # FYI: contains Mkt-RF and RF, but no Mkt

#STEP 2 Get your signal,construct portfolio returns.
# now, here we'd develop some "signal" and then create portfolio rets based on it
# I'm skipping... you figure that out

# I'll pretend I did that by grabbing 5 industry portfolio returns
df_portfolios = ff.FamaFrenchReader('5_Industry_Portfolios', start='1900-01-01').read()[0]
df_portfolios.columns = [f'Port{i+1}' for i in range(len(df_portfolios.columns))] # this is my anticipated portfolio number name scheme

df_portfolios.eval("HmL = Port5-Port1", inplace=True)

# Make each portfolio (except for HmL) excess returns
for col in ['Port1', 'Port2', 'Port3', 'Port4', 'Port5']:
    df_portfolios[col] = df_portfolios[col] - df_factors['RF']

portfolios = df_portfolios.columns.tolist()

df_portfolios